## Data set

In [1]:
import matplotlib.pyplot as plt # 시각화를 위한 라이브러리

import torch # PyTorch 라이브러리
import torch.nn as nn # 모델 구성을 위한 라이브러리
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader # optimizer 설정을 위한 라이브러리

import torchvision # PyTorch의 컴퓨터 비전 라이브러리
import torchvision.transforms as T # 이미지 변환을 위한 모듈
import torchvision.utils as vutils # 이미지를 쉽게 처리하기 위한 유틸리티 모듈

import random

In [2]:
 # seed 고정
import random
import torch.backends.cudnn as cudnn

def random_seed(seed_num):
    torch.manual_seed(seed_num)
    torch.cuda.manual_seed(seed_num)
    torch.cuda.manual_seed_all(seed_num)
    np.random.seed(seed_num)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seed_num)

random_seed(42)

In [4]:
transform_train = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_valid = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                          
])

In [5]:
root = './data'
training_dataset = torchvision.datasets.CIFAR10(root=root, train=True, transform=transform_train, download=True)
test_dataset = torchvision.datasets.CIFAR10(root=root, train=False, transform=transform_valid, download=True)


100%|██████████| 170498071/170498071 [00:38<00:00, 4377194.23it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
num_train, num_val = int(len(training_dataset) * 0.8), int(len(training_dataset) * 0.2)
train_dataset, valid_dataset = torch.utils.data.random_split(training_dataset, [num_train, num_val])
print(len(train_dataset))
print(len(valid_dataset))

40000
10000


In [7]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Pytorch lighting

In [8]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, Trainer, LightningDataModule

import torchmetrics

In [10]:
class classifier(LightningModule):
    def __init__(self, num_classes, dropout_ratio, lr=0.001):
        super().__init__()
        self.learning_rate = lr
        self.accuracy = torchmetrics(task='multiclass', num_classes=num_classes)
        self.criterion = nn.CrossEntropyLoss()
        
         
        self.num_classes = num_classes
        self.dropout_ratio = dropout_ratio
        
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(self.dropout_ratio),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(self.dropout_ratio),
        )
        
        self.fc_layer = nn.Linear(1024,self.num_classes) 
        self.softmax = nn.LogSoftmax(dim=1)
        
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(x.size(0), -1)
        pred = self.fc_layer(out)
        
        return pred 
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
        
        return [optimizer], [scheduler]
    
    def training_step(self, batch, batch_idx):
        images, labels = batch 
        
        outputs = self(images)
        
        loss = self.criterion(outputs, labels)
        acc = self.accuracy(outputs, labels)
        self.log("train_loss", loss, on_step=False, on_epoch=True, logger=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True, logger=True) 
        return loss 
    
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        _, preds = torch.max(outputs, dim=1)
        acc = self.accuracy(preds, labels)
        
        self.log(f"valid_loss", loss, on_step=False, on_epoch=True, logger=True)
        self.log(f"valid_acc", acc ,on_step=False, on_epoch=True, logger=True) 
    
    def test_step(self, batch, batch_idx):
        images, labels = batch 
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        _, preds = torch.max(outputs, dim=1)
        acc = self.accuracy(preds, labels)
        
        self.log(f"test_loss", loss, on_step=False, on_epoch=True)
        self.log(f"test_acc", acc, on_step=False, on_epoch=True)
        
    def predict_step(self, batch, batch_idx):
        images, labels = batch 
        outputs = self(images)
        _, preds = torch.max(outputs, dim=1)

        return preds     